In [1]:
from torch.autograd import Variable
import torch
import onnxruntime
import tensorflow as tf
import onnx2tf

In [4]:
model = torch.load('models/faster_not_jit_rcnn_model.pt', map_location='cpu')
img_size = (640, 640)
batch_size = 1
onnx_model_path = 'model.onnx'
sample_input = Variable(torch.rand((batch_size, 3, *img_size)))

y = model(sample_input)
y

[{'boxes': tensor([[449.8624,  73.7949, 640.0000, 595.1772],
          [ 23.0561,  70.3881, 188.5582, 389.9374],
          [412.3035,  96.0241, 621.5682, 360.9136],
          [443.1876, 274.0103, 630.7805, 539.3550],
          [344.7103,  52.3481, 634.7700, 457.2322],
          [ 10.9015,  94.8681, 287.9455, 560.0727],
          [  2.4115,  56.7143, 136.1261, 593.2604],
          [315.6378, 257.7193, 640.0000, 608.1486],
          [ 20.4525, 212.2023, 189.1365, 552.5846]], grad_fn=<StackBackward0>),
  'labels': tensor([2, 2, 2, 2, 2, 2, 2, 2, 2]),
  'scores': tensor([0.0941, 0.0701, 0.0619, 0.0593, 0.0584, 0.0584, 0.0575, 0.0524, 0.0500],
         grad_fn=<IndexBackward0>)}]

In [3]:
torch.onnx.export(
    model,
    sample_input, 
    onnx_model_path,
    verbose=False,
    input_names=['images'], 
    output_names=['boxes', 'labels', 'scores'],
)

In [5]:
session = onnxruntime.InferenceSession("model.onnx")
onnx_output = session.run(["boxes", 'labels', 'scores'], {"images": torch.rand((1, 3, 640,640)).numpy()})
print("[ONNX] Model Outputs:", [o.name for o in session.get_outputs()])
print("[ONNX] Model Predictions:", onnx_output)

[ONNX] Model Outputs: ['boxes', 'labels', 'scores']
[ONNX] Model Predictions: [array([[447.12375  ,  65.44785  , 640.       , 569.8071   ],
       [  4.3053346,  89.29026  , 179.57637  , 590.9504   ],
       [466.1367   ,  93.82244  , 640.       , 328.62137  ],
       [ 14.015589 , 100.30181  , 303.46906  , 304.37463  ],
       [365.2206   ,  89.331665 , 617.82794  , 363.25873  ]],
      dtype=float32), array([2, 2, 2, 2, 2], dtype=int64), array([0.0831432 , 0.06551051, 0.06000041, 0.05850454, 0.05648219],
      dtype=float32)]


In [13]:
onnx2tf.convert(
    input_onnx_file_path="model.onnx",
    output_folder_path="saved_model",
    copy_onnx_input_output_names_to_tflite=True,
    non_verbose=True,
    output_keras_v3=True,
    param_replacement_file="param_replacement.json"
)

In [14]:
# Now, test the newer TFLite model
interpreter = tf.lite.Interpreter(model_path="saved_model/model_float32.tflite")
interpreter.get_input_details()

[{'name': 'inputs_0',
  'index': 0,
  'shape': array([  1, 640, 640,   3]),
  'shape_signature': array([  1, 640, 640,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
# import tensorflow as tf

# # Load the TensorFlow Lite model
# tflite_model_path = "model/model_float32.tflite"
# interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
# interpreter.allocate_tensors()

# # Get input and output details
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# # Prepare and input data (replace with your actual input data)
# input_data = torch.rand(1, 640, 640, 3)

# # Set the input tensor with your data
# interpreter.set_tensor(input_details[0]['index'], input_data)

# # Run the inference
# interpreter.invoke()

# # Get the output tensor
# output_data = interpreter.get_tensor(output_details[0]['index'])

# # Output data contains the model's predictions
# print("[TFLite] Model Predictions:", output_data)
